# Mobilfunkmasten und Netzabdeckung

In den letzten zwei Jahrzehnten hat sich die Mobilfunktechnologie rasant entwickelt und der Bedarf an Mobilfunkdiensten ist stetig gestiegen. Diese Dienste werden durch ein Netz von Mobilfunkmasten bereitgestellt, die in der Regel von Mobilfunkbetreibern betrieben werden.

Die Errichtung von Mobilfunkmasten ist jedoch mit Kosten verbunden, und die Netzbetreiber müssen sicherstellen, dass die Mobilfunkmasten so platziert werden, dass sie eine optimale Netzabdeckung bieten und gleichzeitig die Anzahl der Mobilfunkmasten so klein wie möglich ist.

![Cell Tower Coverage](https://media.rnztools.nz/rnz/image/upload/s--5-nd_aXq--/c_scale,f_auto,q_auto,w_1050/v1651703353/4LTH50J_image_crop_141730)

## Problemstellung

Ein Mobilfunknetzbetreiber plant die Errichtung einer Reihe von Mobilfunkmasten, um die Bewohner einer bestimmten Stadt mit einem Signal zu versorgen. Die Stadt wurde in einem früheren Schritt in eine Reihe von Regionen unterteilt. Der Betreiber muss nun entscheiden in welchen Regionen er Mobilfunkmasten errichten soll, um die größtmögliche Anzahl von Bewohnern zu erreichen. Die Anzahl der Bewohner in jeder Region ist bekannt und in der folgenden Tabelle aufgeführt:

In [15]:
import pandas as pd

data = [
    ["Tower 0", 1, 1, 0, 0, 0, 1, 0, 0, 0],
    ["Tower 1", 1, 0, 0, 0, 0, 0, 0, 1, 1],
    ["Tower 2", 0, 0, 1, 1, 1, 0, 1, 0, 0],
    ["Tower 3", 0, 0, 1, 0, 0, 1, 1, 0, 0],
    ["Tower 4", 1, 0, 1, 0, 0, 0, 1, 1, 1],
    ["Tower 5", 0, 0, 0, 1, 1, 0, 0, 0, 1]
]

regions = ["Region 1", "Region 2", "Region 3", "Region 4", "Region 5", "Region 6", "Region 7", "Region 8", "Region 9"]

df = pd.DataFrame(data, columns=["Tower"] + regions)
df.set_index("Tower", inplace=True)

pop_data = [523, 690, 420, 1010, 1200, 850, 400, 1008, 950]
pop = pd.Series(
    pop_data, 
    name="Population",
    index=regions
)

pop.to_frame()

,Population
Region 1,523
Region 2,690
Region 3,420
Region 4,1010
Region 5,1200
Region 6,850
Region 7,400
Region 8,1008
Region 9,950


Die Reichweiten der Mobilfunkmasten sind ebenfalls bekannt und in der folgenden Tabelle aufgeführt:

In [16]:
df

,Region 1,Region 2,Region 3,Region 4,Region 5,Region 6,Region 7,Region 8,Region 9
Tower,,,,,,,,,
Tower 0,1,1,0,0,0,1,0,0,0
Tower 1,1,0,0,0,0,0,0,1,1
Tower 2,0,0,1,1,1,0,1,0,0
Tower 3,0,0,1,0,0,1,1,0,0
Tower 4,1,0,1,0,0,0,1,1,1
Tower 5,0,0,0,1,1,0,0,0,1


In [19]:
df_c = df.unstack()

## Modellierung

### Entscheidungsvariablen

Zuerst müssen wir die Entscheidungsvariablen definieren. Die Frage, die wir beantworten müssen ist es, an welchen Stellen wir Mobilfunkmasten errichten sollten. Wir können deswegen für jede Stelle eine binäre Variable definieren, die angibt, ob an dieser Stelle ein Mobilfunkmast errichtet wird oder nicht. Wir definieren die binäre Variable $x_{i}$ als:

$x_{l} = \begin{cases} 1 & \text{wenn ein Mobilfunkmast in Region } l \text{ errichtet wird} \\ 0 & \text{sonst} \end{cases}$

Um die Notation zu vereinfachen, definieren wir einen Index $l \in L$, der die Stellen der Mobilfunkmasten angibt und einen weiteren Index $r \in R$, der die Regionen angibt.

### Parameter

Die Parameter des Problems sind die Anzahl der Bewohner in jeder Region $R_{j}$ und die Abdeckung der Mobilfunkmasten an jeder stelle Region $t \in T$.

* $p_{r}, \quad r \in R$: Anzahl der Bewohner in Region $j$
* $c_{l}, \quad l \in L$: Kosten für die Errichtung eines Mobilfunkmastes an Stelle $l$
* $s_{r,l} \in \{0, 1\}, \quad r \in R, l \in L$: Gibt an, ob die Region $r$ von einem Mobilfunkmast an Stelle $l$ abgedeckt wird.

## Zielfunktion

Das Ziel des Netzbetreibers ist es, die Anzahl der Bewohner zu maximieren, die von den Mobilfunkmasten abgedeckt werden.

$$
\text{max} \sum_{r \in R} \sum_{l \in L} p_{r} \cdot s_{r,l} \cdot x_{l}
$$


## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

We now present a mixed-integer programming (MIP) formulation for the Cell Tower Coverage Problem.

## Model Formulation

### Sets and Indices

$i \in T$: Index and set of potential sites to build a tower.

$j \in R$: Index and set of regions.

$G(T,R,E)$: A bipartite graph defined over the set $T$ of potential sites to build a tower,  the set of regions $R$ that we want to cover, and $E$ is the set of edges, where we have an edge $(i,j) \in E$ if region $j \in R$ can be covered by a tower on location $i \in T$.

### Parameters

$c_{i} \in \mathbb{R}^+$: The cost of setting up a tower at site $i$.

$p_{j} \in \mathbb{N}$: The population at region $j$.

### Decision Variables

$covered_{j} \in \{0, 1 \}$: This variable is equal to 1 if region $j$ is covered; and 0 otherwise.

$build_{i} \in \{0, 1 \}$: This variable is equal to 1 if tower $i$ is built; and 0 otherwise.

### Objective Function(s)

- **Population covered**. We seek to maximize the total population covered by the towers.

\begin{equation}
\text{Max} \quad Z = \sum_{j \in R} p_{j} \cdot covered_{j}
\tag{0}
\end{equation}

### Constraints

- **Coverage**. For each region $j \in R$ ensure that at least one tower that covers a region must be selected.

\begin{equation}
\sum_{(i,j) \in E} build_{i} \geq covered_{j} \quad \forall j \in R
\tag{1}
\end{equation}

- **Budget**. We need to ensure that the total cost of building towers do not exceed the allocated budget.

\begin{equation}
\sum_{i \in T} c_{i} \cdot build_{i} \leq \text{budget}
\tag{2}
\end{equation}

## Python Implementation

This example considers a bipartite graph for 6 towers and 9 regions. The following table illustrates which regions (columns) are covered by each cell tower site (rows).

| <i></i> | Region 0 | Region 1 | Region 2 | Region 3 | Region 4 | Region 5 | Region 6 | Region 7 | Region 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |  --- |
| Tower 0 | 1 | 1 | - | - | - | 1 | - | - |  - |
| Tower 1 | 1 | - | - | - | - | - | - | 1 |  1 |
| Tower 2 | - | - | 1 | 1 | 1 | - | 1 | - |  - |
| Tower 3 | - | - | 1 | - | - | 1 | 1 | - |  - |
| Tower 4 | 1 | - | 1 | - | - | - | 1 | 1 |  1 |
| Tower 5 | - | - | - | 1 | 1 | - | - | - |  1 |

The population at each region is stated in the following table.

| <i></i> | Region 0 | Region 1 | Region 2 | Region 3 | Region 4 | Region 5 | Region 6 | Region 7 | Region 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Population | 523 | 690 | 420 | 1010 | 1200 | 850 | 400 | 1008 | 950 |

The cost to build a cell tower at each location site is stated inthe following table.

| <i></i> | Cost (millions of USD) |
| --- | --- |
| Tower 0 | 4.2 |
| Tower 1 | 6.1 |
| Tower 2 | 5.2 |
| Tower 3 | 5.5 |
| Tower 4 | 4.8 |
| Tower 5 | 9.2 |

The allocated budget is $\$20,000,000$.

We now import the Gurobi Python Module. Then, we initialize the data structures with the given data.


In [ ]:
%pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.0.0 and Python 3.7.0

# Parameters
budget = 20
regions, population = gp.multidict({
    0: 523, 1: 690, 2: 420,
    3: 1010, 4: 1200, 5: 850,
    6: 400, 7: 1008, 8: 950
})

sites, coverage, cost = gp.multidict({
    0: [{0,1,5}, 4.2],
    1: [{0,7,8}, 6.1],
    2: [{2,3,4,6}, 5.2],
    3: [{2,5,6}, 5.5],
    4: [{0,2,6,7,8}, 4.8],
    5: [{3,4,8}, 9.2]
})


### Model Deployment

We now determine the model for the Cell Tower Coverage Problem, by defining the decision variables, constraints, and objective function. Next, we start the optimization process and Gurobi finds the plan to build towers that maximizes  the coverage of the population given the budget allocated.

In [ ]:
# MIP  model formulation
m = gp.Model("cell_tower")

build = m.addVars(len(sites), vtype=GRB.BINARY, name="Build")
is_covered = m.addVars(len(regions), vtype=GRB.BINARY, name="Is_covered")

m.addConstrs((gp.quicksum(build[t] for t in sites if r in coverage[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")
m.addConstr(build.prod(cost) <= budget, name="budget")

m.setObjective(is_covered.prod(population), GRB.MAXIMIZE)

m.optimize()

Using license file c:\gurobi\gurobi.lic
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 15 columns and 36 nonzeros
Model fingerprint: 0xfa0fabb2
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolved: 6 rows, 10 columns, 21 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 7.051000e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7051.0000000 7051.00000  0.00%     -    0s

Explored 0 nodes (1 simplex i

## Analysis
The result of the optimization model shows that the maximum population that can be covered with the $\$20,000,000$ budget is 7,051 people. Let's see the solution that achieves that optimal result.

### Cell Tower Build Plan

This plan determines at which site locations to build a cell tower.

In [ ]:
# display optimal values of decision variables

for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        print(f"\n Build a cell tower at location Tower {tower}.")


 Build a cell tower at location Tower 0.

 Build a cell tower at location Tower 2.

 Build a cell tower at location Tower 4.


### Demand Fulfillment Metrics

- **Coverage**: Percentage of the population covered by the cell towers built.

In [ ]:
# Percentage of the population covered by the cell towers built is computed as follows.

total_population = 0

for region in range(len(regions)):
    total_population += population[region]

coverage = round(100*m.objVal/total_population, 2)

print(f"\n The population coverage associated to the cell towers build plan is: {coverage} %")


 The population coverage associated to the cell towers build plan is: 100.0 %


### Resources Utilization Metrics

- **Budget consumption**: Percentage of the budget allocated to build the cell towers.

In [ ]:
# Percentage of budget consumed to build cell towers

total_cost = 0

for tower in range(len(sites)):
    if (abs(build[tower].x) > 0.5):
        total_cost += cost[tower]*int(build[tower].x)

budget_consumption = round(100*total_cost/budget, 2)

print(f"\n The percentage of budget consumed associated to the cell towers build plan is: {budget_consumption} %")


 The percentage of budget consumed associated to the cell towers build plan is: 71.0 %


##  Conclusion

In this example, we address the problem of building cell towers to provide signal coverage  to the largest number of people while satisfying a budget constraint. We learned how to formulate the problem as a MIP model. Also, we learned how to implement the MIP model formulation and solve it using the Gurobi Python API.

## References

[1] Richard Church and Charles R. Velle. "The Maximal Covering Location Problem". Papers in Regional Science, 1974, vol. 32, issue 1, 101-118.

[2] Tail Assignment Problem. https://www.gurobi.com/case_study/air-france-tail-assignment-optimization/

Copyright © 2020 Gurobi Optimization, LLC